# **``TF-IDF``**

![Alt texts](1.png)

## **Imports**

In [ ]:
import math
import nltk
from nltk.corpus import treebank
from collections import defaultdict
import numpy as np
import pandas as pd
import re

In [ ]:
class TFIDFVectorizer:
    def __init__(self, preprocessing=None):
        self.vocabulary = set()
        self.document_frequency = defaultdict(int)
        self.idf_values = {}
        self.documents = []
        self.file_ids = []
        self.preprocessing = self._default_preprocessor

    def _default_preprocessor(self, text):
        # Якщо текст - список, перетворюємо на рядок
        if isinstance(text, list):
            text = ' '.join(text)
        
        # Перетворення на нижній регістр та видалення зайвих символів
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text

    def _preprocess_text(self, text):
        # Якщо текст - список слів, перетворюємо на рядок
        if isinstance(text, list):
            text = ' '.join(text)
        
        # Якщо текст є об'єктом StreamBackedCorpusView
        try:
            text = ' '.join(text)
        except:
            pass
        
        # Застосовуємо додаткову обробку 
        return self.preprocessing(text)

    def _counter(self, document, term):
        return document.split().count(term)

    def _compute_document_frequency(self):
        self.document_frequency.clear()
        self.vocabulary.clear()
        
        for document in self.documents:
            unique_terms = set(document.split())
            for term in unique_terms:
                self.document_frequency[term] += 1
                self.vocabulary.add(term)

    def _compute_idf(self):
        document_count = len(self.documents)
        self.idf_values = {
            term: math.log((document_count + 1) / (freq + 1))
            for term, freq in self.document_frequency.items()
        }

    def compute_tf(self, document, term):
        count = self._counter(document, term)
        return 1 + math.log(count) if count > 0 else 0

    def compute_tf_idf(self, document, term):
        tf = self.compute_tf(document, term)
        idf = self.idf_values.get(term, 0)
        return tf * idf

    def load_treebank_corpus(self, preprocessing=None):
        # Завантаження необхідних ресурсів NLTK
        #nltk.download('treebank')
        
        # Очищення попередніх даних
        self.documents.clear()
        self.file_ids.clear()
        
        # Встановлення функції попередньої обробки
        if preprocessing:
            self.preprocessing = preprocessing
        
        # Обробка кожного файлу як окремого документа
        for file_id in treebank.fileids():
            # Отримання слів з файлу та перетворення на рядок
            words = treebank.words(file_id)
            processed_doc = self._preprocess_text(words)
            self.documents.append(processed_doc)
            self.file_ids.append(file_id)
        
        return self

    def fit(self):
        # Обчислення частоти документів
        self._compute_document_frequency()
        
        # Обчислення inverse document frequency
        self._compute_idf()
        
        return self

    def transform(self):
        # Перевірка, чи була модель натренована
        if not self.idf_values:
            raise ValueError("Спочатку викличіть метод fit()")
        
        # Створення матриці TF-IDF
        tfidf_matrix = np.zeros((len(self.documents), len(self.vocabulary)))
        
        # Перетворення vocabulary в список для індексації
        terms_list = list(self.vocabulary)
        
        # Обчислення TF-IDF для кожного документа та терміну
        for doc_idx, doc in enumerate(self.documents):
            for term_idx, term in enumerate(terms_list):
                tfidf_matrix[doc_idx, term_idx] = self.compute_tf_idf(doc, term)
        
        return tfidf_matrix, terms_list

    def fit_transform(self):
        return self.fit().transform()

    def analyze_tfidf_matrix(self, tfidf_matrix, terms_list):
        # Підрахунок ненульових значень для кожного терміну
        non_zero_counts = np.count_nonzero(tfidf_matrix, axis=0)
        
        # Обчислення середніх значень TF-IDF для кожного терміну
        mean_tfidf = np.mean(tfidf_matrix, axis=0)
        
        # Створення DataFrame для аналізу
        analysis_df = pd.DataFrame({
            'term': terms_list,
            'non_zero_docs': non_zero_counts,
            'mean_tfidf': mean_tfidf
        })
        
        # Сортування за середнім TF-IDF у спадному порядку
        return analysis_df.sort_values('mean_tfidf', ascending=False)

# Приклад використання з розширеним налагодженням
if __name__ == "__main__":
    # Створення векторизатора
    vectorizer = TFIDFVectorizer()
    
    # Завантаження корпусу Treebank
    vectorizer.load_treebank_corpus()
    
    # Обчислення матриці TF-IDF
    tfidf_matrix, terms = vectorizer.fit_transform()
    
    # Виведення результатів
    print("Розмір матриці TF-IDF:", tfidf_matrix.shape)
    print("Загальна кількість унікальних термінів:", len(terms))
    
    # Аналіз матриці TF-IDF
    analysis = vectorizer.analyze_tfidf_matrix(tfidf_matrix, terms)
    
    # Виведення топ-20 термінів за TF-IDF
    print("\nТоп-20 термінів за TF-IDF:")
    print(analysis.head(20))
    
    # Додаткова діагностика
    print("\nКількість документів з ненульовими TF-IDF значеннями:")
    print(np.count_nonzero(tfidf_matrix, axis=1))

Розмір матриці TF-IDF: (199, 11036)
Загальна кількість унікальних термінів: 11036

Топ-20 термінів за TF-IDF:
          term  non_zero_docs  mean_tfidf
3461        mr             82    0.735963
9273        he             80    0.696529
10070     that            127    0.692223
10850     says             32    0.688334
5404        us             67    0.672095
2460      they             83    0.669513
3089      have             97    0.661569
3295       are             98    0.654389
5264        at            111    0.654083
8323     their             55    0.651955
2580       but             97    0.649064
4409     would             76    0.638284
7580    market             53    0.625163
8726   million            109    0.624915
10364       as            114    0.621238
243         or             99    0.618202
2547        nt            102    0.617928
10969     more             80    0.613188
8889        t2            109    0.609510
7689        t1            135    0.609084

Кількіс